In [14]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import theano
from PIL import Image
from numpy import *

In [16]:
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

In [17]:
img_rows, img_cols = 100,100

In [18]:
img_channels = 1

In [19]:
path1='/home/bharath/car_shape/side_view'
path2='/home/bharath/car_shape/front_view'
#path3='/home/bharath/car_model/verna'
path='/home/bharath/car_shape/resize_model'

In [20]:
listing1 = os.listdir(path1) 
listing2 = os.listdir(path2)
#listing3 = os.listdir(path3) 

In [21]:
for file in listing1:
    im = Image.open(path1 + '/' + file)  
    img = im.resize((img_rows,img_cols))
    gray = img.convert('L')          
    img.save(path +'/' +  file, "JPEG")

In [22]:
for file in listing2:
    im = Image.open(path2 + '/' + file)  
    img = im.resize((img_rows,img_cols))
    gray = img.convert('L')          
    img.save(path +'/' +  file, "JPEG")

In [84]:
for file in listing3:
    im = Image.open(path3 + '/' + file)  
    img = im.resize((img_rows,img_cols))
    gray = img.convert('L')          
    img.save(path +'/' +  file, "JPEG")

In [23]:
from scipy import ndimage

In [24]:

immatrix=[]
for imageName in sorted(os.listdir("/home/bharath/car_shape/resize_model/")):
    immatrix.append(ndimage.imread("/home/bharath/car_shape/resize_model/%s"%(imageName)))

In [25]:
imp=np.array(immatrix)

In [26]:

print imp.shape

(157, 100, 100, 3)


In [27]:
label=np.ones((157,),dtype = int)
label[0:79]=0
label[79:157]=1
#label[199:274]=2

In [28]:
data,Label = shuffle(imp,label, random_state=2)
train_data = [data,Label]

In [29]:
#batch_size to train
batch_size = 32
# number of output classes
nb_classes = 2
# number of epochs to train
nb_epoch = 20

In [30]:
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 1

In [31]:
(X, y) = (train_data[0],train_data[1])

print X.shape

(157, 100, 100, 3)


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)


#X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
#X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255


In [33]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

('X_train shape:', (125, 100, 100, 3))
(125, 'train samples')
(32, 'test samples')


In [34]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [35]:
print(nb_filters, nb_conv, img_rows, img_cols)

(32, 1, 100, 100)


In [36]:
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(img_rows, img_cols,3)))
convout1 = Activation('relu')
model.add(convout1)
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
convout2 = Activation('relu')
model.add(convout2)
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(1))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adadelta')

In [37]:
hist = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=48,
              show_accuracy=True, verbose=1, validation_data=(X_test, Y_test))

/home/bharath/anaconda2/lib/python2.7/site-packages/keras/models.py:610: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Train on 125 samples, validate on 32 samples
Epoch 1/48
125/125 [==============================] - 0s - loss: 0.5559 - acc: 0.7120 - val_loss: 0.5980 - val_acc: 0.6875
Epoch 2/48
125/125 [==============================] - 0s - loss: 0.7001 - acc: 0.5920 - val_loss: 0.4303 - val_acc: 0.8438
Epoch 3/48
125/125 [==============================] - 0s - loss: 0.4430 - acc: 0.8080 - val_loss: 0.4685 - val_acc: 0.8125

In [38]:
scores = model.evaluate(X_test, Y_test, verbose=0)

In [39]:

print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 93.75%


In [45]:
def names(a):
    if(a==0):
        return 'SIDE VIEW'
    elif(a==1):
        return 'FRONT VIEW'
    

In [41]:
itest = Image.open('/home/bharath/car_shape/abc.jpeg')  
imtest = itest.resize((img_rows,img_cols))      
imtest.save('/home/bharath/car_shape/file.jpg' )

In [42]:
imgs=[]
imgs.append(ndimage.imread("/home/bharath/car_shape/file.jpg"))

In [43]:
ba=np.asarray(imgs)
ab=model.predict_classes(ba)

1/1 [==============================] - 0s


In [46]:
asd=names(ab)

In [47]:
print asd

FRONT VIEW


In [48]:
itest = Image.open('/home/bharath/car_shape/abc1.jpeg')  
imtest = itest.resize((img_rows,img_cols))      
imtest.save('/home/bharath/car_shape/file1.jpg' )

In [49]:
imgs=[]
imgs.append(ndimage.imread("/home/bharath/car_shape/file1.jpg"))

In [50]:
ba=np.asarray(imgs)
ab=model.predict_classes(ba)

1/1 [==============================] - 0s


In [51]:
asd=names(ab)

In [52]:
print asd

SIDE VIEW
